In [1]:
import tensorflow as tf 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# custom
from utils.utils import load_local_dataset_train_test, load_local_dataset, load_local_dataset_tf
from utils.constants import SOURCE_DATASETS, TARGET_DATASETS


In [2]:

TARGET_SIZE = (300, 300)
# BATCH_SIZE = 128
BATCH_SIZE = 8

dataset_name, img_format, num_classes, initial_res = TARGET_DATASETS[0]

# 
# path = "/Users/felixgerschner/git/ai-prototype/data/target/mechanicalseals_fulllight"
# path = "/Users/felixgerschner/git/ai-prototype/data/source/miniimagenet"
# load_local_dataset_tf(path, target_size=TARGET_SIZE)


path = "/Users/felixgerschner/git/ai-prototype/data/source/dagm"
load_local_dataset_tf(path, target_size=TARGET_SIZE)


# path = "/Users/felixgerschner/git/ai-prototype/data/source/caltech101"
# load_local_dataset_tf(path, target_size=TARGET_SIZE)


# class_names = data.class_names
# print(class_names)

# for images, labels in data.take(1):
#     print(images.shape, " ", labels.shape)
 
#     print(labels[0])
#     plt.imshow(images[10].numpy().astype("uint8"))
#     plt.axis("off")


# X, y = load_local_dataset(path, img_format, target_size=TARGET_SIZE)


Found 60000 files belonging to 100 classes.
Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-02-02 20:28:28.419552: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-02 20:28:28.419696: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<ShuffleDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.float32, name=None))>

In [3]:
# data = tf.keras.utils.image_dataset_from_directory(
#   path,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=TARGET_SIZE,
#   batch_size=BATCH_SIZE,
#   interpolation="area"
# )


# for images, labels in data.take(1):
#     print(images.shape, " ", labels.shape)
 
#     print(labels[1])
#     plt.imshow(images[0].numpy().astype("uint8"))
#     plt.axis("off")


In [4]:
# plt.imshow(X[4].astype("uint8"))
# plt.axis("off")
# X[4].shape

In [5]:
# from models.tf_models.VGG16_model import VGG16_model


# model = VGG16_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [6]:
# from models.tf_models.ResNet101_model import ResNet101_model


# model = ResNet101_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [7]:
# from models.tf_models.DenseNet121_model import DenseNet121_model


# model = DenseNet121_model("imagenet", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [8]:
# from models.tf_models.MobileNet_model import MobileNet_model


# model = MobileNet_model("mechanicalseal_fulllight", input_shape=(350, 350, 3), 
#                 num_classes=2, build_pre_model=True)

# model.fit_and_save_pre_model(X, y)

In [9]:
# import matplotlib.pyplot as plt

# plt.imshow(X[2], cmap="binary")
# plt.show()

In [10]:
# from tensorflow.keras.applications.vgg16 import VGG16

# model = VGG16(weights="imagenet", include_top=True)

# model.summary()

In [11]:
import tensorflow as tf 


from sklearn.model_selection import train_test_split

# custom
from utils.utils import load_local_dataset, load_local_dataset_train_test, preload_from_directory_old

# path = "/Users/felixgerschner/git/ai-prototype/data/source/miniimagenet" # CRASHES KERNEL
# path = "/Users/felixgerschner/git/ai-prototype/data/source/caltech101" # Works
# path = "/Users/felixgerschner/git/ai-prototype/data/source/dagm" # Works
# path = "/Users/felixgerschner/git/ai-prototype/data/target/mechanicalseals_fulllight" # Works

# X, y = load_local_dataset(path, ".JPEG", input_shape=None) # CRASHES KERNEL
# X, y = load_local_dataset(path, ".jpg", input_shape=(350, 350))
# X, y = load_local_dataset(path, ".PNG", input_shape=(350, 350))
# X, y = load_local_dataset(path, ".png", input_shape=(350, 350))
# X_train, y_train, X_test, y_test = load_local_dataset_train_test(path, ".png", input_shape=(350, 350))



In [12]:
# X.shape
X_train.shape, " ", X_test.shape

NameError: name 'X_train' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X[2523].astype("uint8"))
plt.show()
y[2523]